In [14]:
import os
import json
import pandas as pd

In [17]:
def clean_value(x):
    import pandas as pd
    if pd.isna(x):  # NaN → None
        return None
    if hasattr(x, "item"):
        x = x.item()
    return str(x)   # всё приводим к строке

In [18]:
DATA_DIR = "./data"

# Выходной файл
OUTPUT_FILE = "./data/train.jsonl"

dataset = []


for fname in os.listdir(DATA_DIR):

    if fname.endswith(" simple.csv"):
        # Находим исходный файл
        base_name = fname.replace(" simple.csv", ".csv")
        simple_path = os.path.join(DATA_DIR, fname)
        input_path = os.path.join(DATA_DIR, base_name)

        if not os.path.exists(input_path):
            print(f"Не найден входной файл для {fname}")
            continue

        # Загружаем оба файла
        df_in = pd.read_csv(input_path, sep=";", encoding="utf-8-sig", header=None)
        df_out = pd.read_csv(simple_path, sep=";", encoding="utf-8")

        # Проверяем размерность
        if len(df_in) != len(df_out):
            print(f"Предупреждение: строки не совпадают ({base_name})")

        for i in range(min(len(df_in), len(df_out))):
            input_row = [clean_value(x) for x in df_in.iloc[i].tolist()]
            output_row = {k: clean_value(v) for k, v in df_out.iloc[i].to_dict().items()}

            dataset.append({
                "input": input_row,   # теперь всегда список строк
                "output": output_row  # словарь со строковыми значениями
            })

# Сохраняем в JSONL
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    for row in dataset:
        f.write(json.dumps(row, ensure_ascii=False) + "\n")

print(f"Датасет сохранён в {OUTPUT_FILE}, всего пар: {len(dataset)}")

Датасет сохранён в ./data/train.jsonl, всего пар: 169
